# **Importamos las dependencias**

Estoy importando todas las dependencias de mis cuadernos, despues las depuro

In [48]:
# --- Importaciones para Manejo de Archivos y Directorios ---
import os
import sys
import subprocess
import re

import math

# --- Importamos Vertex AI ---
import vertexai


# --- Importaciones para Visualización y Formato de Texto ---

# Se utiliza para mostrar contenido enriquecido, como texto con formato Markdown,
# directamente en entornos como Jupyter Notebooks o IPython.
from IPython.display import Markdown, display

# Importa la clase `Markdown` de la biblioteca `rich`, que sirve para renderizar
# Markdown con formato avanzado en la terminal. Se le da un alias `RichMarkdown`
# para evitar conflictos de nombre con la importación anterior.
from rich.markdown import Markdown as RichMarkdown


# --- Importaciones para el Modelo Generativo de Vertex AI ---

# Importa las clases necesarias del SDK de Vertex AI para interactuar con los modelos generativos.
# - GenerationConfig: Para configurar los parámetros de la respuesta (ej. temperatura, top_p).
# - GenerativeModel: La clase principal para cargar y usar un modelo generativo como Gemini.
# - Image: Para manejar y enviar imágenes como parte de la entrada al modelo (enfoque multimodal).
from vertexai.generative_models import GenerationConfig, GenerativeModel, Image

In [4]:
# --- Para el proceso de datos y visualización ---
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import time


# --- Para manejar las variables de entorno ---
import os
from dotenv import load_dotenv


# --- Desactiva las advertencias de asignaciones encadenadas en pandas para evitar mensajes de warning al modificar DataFrames.
pd.options.mode.chained_assignment = None  # default='warn'


# --- Dependencias de Vertex AI ---
import vertexai                                              # Importa el módulo principal de Vertex AI.
from vertexai import init                                    # Inicializa Vertex AI con las credenciales y configuraciones necesarias.
from vertexai.vision_models import Image as VMImage          # Importa la clase Image de Vertex AI para manejar imágenes.
from vertexai.vision_models import MultiModalEmbeddingModel  # Importa el modelo de embeddings multimodales de Vertex AI para procesar imágenes y videos.
from vertexai.vision_models import Video                     # Clase para manejar archivos de video en Vertex AI.
from vertexai.vision_models import VideoSegmentConfig        # Configuración para segmentar videos
from vertexai.generative_models import GenerativeModel       # Importa la clase para modelos generativos, como Gemini.
from vertexai.generative_models import Part                  # Importa la clase Part para manejar partes de un mensaje, como texto o imágenes.


# --- Para conectarse y consultar un endpoint de búsqueda vectorial (Vector Search) en Vertex AI. 
from google.cloud.aiplatform.matching_engine import MatchingEngineIndexEndpoint 

# --- Para acceder a los buckets de Google Cloud Storage y manejar archivos.
from google.cloud import storage


# --- Dependencias para poder visualizar ---
from IPython.display import Video as MVideo                  # Permite mostrar videos directamente en celdas de Jupyter Notebook.
from IPython.display import HTML                             # Permite mostrar contenido HTML en celdas de Jupyter Notebook.
from IPython.display import Image as ImageByte               # Permite mostrar imágenes en el notebook (renombrado como ImageByte para evitar conflictos de nombres).
from IPython.display import display                          # Función general para mostrar objetos en el notebook.
from sklearn.metrics.pairwise import cosine_similarity       # Función para calcular la similitud coseno entre vectores, útil para comparar embeddings.

# **Configuración de credenciales**

In [5]:
# --- Carga las Variables de Entorno ---
load_dotenv()

PROJECT_ID = os.getenv("PROJECT_ID")                        # ID del proyecto de Google Cloud
LOCATION = os.getenv("LOCATION")                            # Región donde se encuentran los recursos de Vertex AI
INDEX_ID = os.getenv("INDEX_ID")                            # ID del índice de búsqueda vectorial en Vertex AI
ENDPOINT_ID = os.getenv("ENDPOINT_ID")                      # ID del endpoint de búsqueda vectorial en Vertex AI
BUCKET_NAME = os.getenv("BUCKET_NAME")                      # Nombre del bucket de Google Cloud Storage donde se almacenan los videos
VIDEO_FOLDER_PATH = os.getenv("VIDEO_FOLDER_PATH")          # Ruta del folder dentro del bucket donde se encuentran los videos

# Verificamos que las variebles de entorno esten bien
# print(f"Proyecto: {PROJECT_ID}, Ubicación: {LOCATION}, Índice: {INDEX_ID}, Endpoint: {ENDPOINT_ID}, Bucket: {BUCKET_NAME}, Ruta de Videos: {VIDEO_FOLDER_PATH}")

python-dotenv could not parse statement starting at line 3
python-dotenv could not parse statement starting at line 4
python-dotenv could not parse statement starting at line 5
python-dotenv could not parse statement starting at line 7
python-dotenv could not parse statement starting at line 15
python-dotenv could not parse statement starting at line 16
python-dotenv could not parse statement starting at line 17
python-dotenv could not parse statement starting at line 19
python-dotenv could not parse statement starting at line 28
python-dotenv could not parse statement starting at line 29
python-dotenv could not parse statement starting at line 30
python-dotenv could not parse statement starting at line 32
python-dotenv could not parse statement starting at line 33


In [6]:
# Iniciamos Vertex AI con el proyecto y la ubicación especificados.
init(project = PROJECT_ID, location = LOCATION)

# **Configuración de Vertex AI**

In [7]:
# --- Inicialización del LLM ---
model = GenerativeModel('gemini-2.5-flash-lite-preview-06-17') # Debo probar con varios modelos, para ver cual tiene mejor rendimiento.

# --- Carga del Modelo de Embeddings Multimodal ---
embedding_model = MultiModalEmbeddingModel.from_pretrained("multimodalembedding@001")

# **Funciones**

In [128]:
# --- Para hacerle embeddings a un determinado texto ---

def text_embedding(text):
    """
    Función para obtener el embedding de un texto utilizando el modelo de embeddings multimodal.

    Args:
        text (str): El texto del cual se desea obtener el embedding.

    Returns:
        list: Un vector de embedding que representa el texto.
    """
    embedding = embedding_model.get_embeddings(contextual_text=text)

    return embedding.text_embedding


In [130]:
# --- Función para buscar los embeddings mas cercanos ---

def busqueda_vectorial(emb, res = 5):
    """
    Realiza una búsqueda vectorial utilizando un texto de consulta y devuelve los resultados más relevantes.

    Args:
        emb (list): Lista de embeddings del texto de consulta.
        res (int): Número de resultados a devolver. Por defecto es 5.

    Returns:
        list: Lista de resultados relevantes encontrados en el índice.
    """

    print('Nos conectamos al endpoint del índice de búsqueda vectorial...')
    index_endpoint = MatchingEngineIndexEndpoint(index_endpoint_name = INDEX_ID)

    print(f'Buscando los {res} resultados más relevantes para la consulta...')
    query = index_endpoint.find_neighbors(

        deployed_index_id=ENDPOINT_ID,  # ID del índice desplegado
        queries=[emb],                  # Lista de embeddings de consulta
        num_neighbors=res               # Número de vecinos a devolver

    )

    return query

In [ ]:
# --- Función para obtener el nombre del clip a partír de los índices de una consulta ---

def get_clip_name(lista):
    """
    La función de 'busqueda_vectorial' nos devuelve una lista de índices, aquí construímos los nombres
    de los respectivos clips de esos segmentos
    
    Args:
        lista (list): Lista de índices.

    Returns:
        lista (list): Lista del nombre de los clips, ordenada por el número de clip
    """


    nombres_de_videos = []

    for item in lista:

        # Extraemos el número del índice
        num = int(re.search(r'\d+', item.id).group())

        # Encontramos el número de clip
        numero_de_clip = math.floor(num / 4)

        # Construimos el nombre del video
        def nombre_clip(n):
            if len(str(n)) == 1:
                return f'mexicosta_segment_00{n}.mkv'
            elif len(str(n)) == 2:
                return f'mexicosta_segment_0{n}.mkv'
            else:
                return f'mexicosta_segment_{n}.mkv'

        # Si la división no es exacta, agregamos el clip actual y el siguiente
        if num % 4 != 0:
            nombre_actual = nombre_clip(numero_de_clip)
            nombre_siguiente = nombre_clip(numero_de_clip + 1)
            nombres_de_videos.append(nombre_actual)
            nombres_de_videos.append(nombre_siguiente)
        else:
            nombre_actual = nombre_clip(numero_de_clip)
            nombres_de_videos.append(nombre_actual)


        # Quitamos los repetidos en la lista
        nombres_de_videos = set(nombres_de_videos)
        nombres_de_videos = list(nombres_de_videos)

    return sorted(
        nombres_de_videos,
        key = lambda x: int(re.search(r'(\d+)', x).group())
    )

In [ ]:
# --- Función para reconstruír la url pública

def get_public_url_from_gcs(gcs_uri: str) -> str:
    """
    Convierte una URI de Google Cloud Storage (gs://bucket/archivo) a una URL pública HTTP.

    Args:
        gcs_uri (str): URI de Google Cloud Storage.

    Returns:
        str: URL pública accesible desde el navegador.
    """
    return gcs_uri.replace("gs://", "https://storage.googleapis.com/").replace(
        " ", "%20"
    )


In [115]:
def buscar_videos_similares(texto_de_la_pregunta, cantidad_de_resultados=5):

    """
    Hacemos el embedding de la pregunta, y buscamos en Vector Search los videos más similares.
    Devolvemos las URIs de los videos encontrados.

    Args:
        texto_de_la_pregunta (str): La pregunta o consulta para buscar videos similares.
        cantidad_de_resultados (int): Número de resultados a devolver. Por defecto es 5.

    Returns:
        list: Una lista de URIs de los videos encontrados que son similares a la pregunta.
    """


    print(f'--- 1.- Hacemos el embedding de la pregunta: {texto_de_la_pregunta} ---')

    # Hacemos el embedding de la pregunta
    try:
        vector_de_la_pregunta = text_embedding(texto_de_la_pregunta)

    except Exception as e:
        print(f'Error al hacer el embedding de la pregunta: {e}')
        return []
    
    print('Fin del paso 1.\n')
    


    print(f'\n --- 2.- Nos conectamos a Vector Search ---')

    index_endpoint = MatchingEngineIndexEndpoint(
        INDEX_ID
    )

    print('Fin del paso 2.\n')



    print(f'\n --- 3.- Buscamos los {cantidad_de_resultados} videos más similares a la pregunta ---')

    try:
        # Buscamos usando la técnica de 'nearest neighbors'
        response = busqueda_vectorial(
            emb=vector_de_la_pregunta,         # Usamos el embedding de la pregunta
            res=cantidad_de_resultados         # Número de vecinos a buscar
        )

        print(f'Obtuvimos los siguientes índices:\n {response}')

        # Extraemos las URIs de los videos encontrados
        uris_completas = []

        if response and response[0]:
            for neighbor in response[0]:
                segment_id_base = neighbor.id # Nos da por ejemplo: "mexicosta_segment_1"

                # Extraemos el prefijo y el número del ID
                match = re.match(r"(.*_)(\d+)", segment_id_base)
                if match:
                    prefijo = match.group(1) # "mexicosta_segment_"
                    numero = match.group(2)  # "1" 
                    
                    # Formateamos el número a 3 dígitos con ceros a la izquierda (ej. "001")
                    numero_formateado = numero.zfill(3)
                    
                    # Reconstruimos el nombre del archivo final
                    nombre_archivo_final = f"{prefijo}{numero_formateado}"
                    
                    # Creamos la URI completa con la extensión .mkv
                    uri = f"gs://{BUCKET_NAME}/{VIDEO_FOLDER_PATH}/{nombre_archivo_final}.mkv"
                    uris_completas.append(uri)
                
    
    except Exception as e:
        print(f'Error al buscar los videos similares: {e}')
        return []

In [116]:
def analizar_fragmentos_con_gemini(
    pregunta_original: str,
    uris_de_videos: list[str] ) -> str:

    """
    Toma la pregunta y una lista de URIs de video, las envía a Gemini
    y devuelve la respuesta generada por el modelo.
    """
    if not uris_de_videos:
        return "No se encontraron videos relevantes para analizar."

    print(f"\nPASO 3.1: Preparando {len(uris_de_videos)} videos para enviar a Gemini...")
    
    # Convertimos cada URI de video en un objeto 'Part' que Gemini entiende
    video_parts = [Part.from_uri(uri, mime_type="video/mkv") for uri in uris_de_videos]

    # Construimos el prompt para el modelo
    prompt_completo = [
        "Eres un asistente experto en análisis de video.",
        "Tu tarea es analizar los siguientes fragmentos de video que te proporciono y responder a la pregunta del usuario de la forma más detallada posible basándote ÚNICAMENTE en el contenido de estos videos.",
        "\n---",
        "PREGUNTA DEL USUARIO:",
        pregunta_original,
        "\n---",
        "FRAGMENTOS DE VIDEO A ANALIZAR:",
        *video_parts
    ]

    print("PASO 3.2: Enviando la solicitud a Gemini... (Esto puede tardar un poco)")
    try:
        # Enviamos el prompt completo al modelo generativo
        response = model.generate_content(prompt_completo)
        return response.text
    except Exception as e:
        return f"Ocurrió un error al contactar con el modelo Gemini: {e}"

# **Pruebas**

In [117]:
if __name__ == "__main__":
    pregunta_del_usuario = "Caliente"
    print("--- INICIANDO PROCESO DE RAG MULTIMODAL ---\n\n")
    uris_relevantes = buscar_videos_similares(pregunta_del_usuario, 10)
    respuesta_final = analizar_fragmentos_con_gemini(pregunta_del_usuario, uris_relevantes)
    print("\n--- RESPUESTA FINAL DE GEMINI ---")
    print(respuesta_final)

--- INICIANDO PROCESO DE RAG MULTIMODAL ---


--- 1.- Hacemos el embedding de la pregunta: Caliente ---
Fin del paso 1.


 --- 2.- Nos conectamos a Vector Search ---
Fin del paso 2.


 --- 3.- Buscamos los 10 videos más similares a la pregunta ---
Nos conectamos al endpoint del índice de búsqueda vectorial...
Buscando los 10 resultados más relevantes para la consulta...
Obtuvimos los siguientes índices:
 [[MatchNeighbor(id='mexicosta_segment_1090', distance=0.14725038409233093, sparse_distance=None, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[], sparse_embedding_values=[], sparse_embedding_dimensions=[]), MatchNeighbor(id='mexicosta_segment_162', distance=0.14469942450523376, sparse_distance=None, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[], sparse_embedding_values=[], sparse_embedding_dimensions=[]), MatchNeighbor(id='mexicosta_segment_905', distance=0.13280153274536133, sparse_distance=None, feature_vector=[], crowding_tag='0', re

# **Borrar**